# [ **RUN ALWAYS** ] Import, initialize

In [1]:
import math
import geemap
import ee
import configparser as cp
import io
import ast
import subprocess

from classification import classifyAndAssess
from featuresForModeling import generateFeatures

ee.Initialize(project = "ee-app-tr-condit-1540393218181")

# [ **RUN ALWAYS** ] Check that features available via config file matches those in the set of feature rasters

Each feature is created separately, in the `featuresForModeling` module, and exported as EE assets. The names of each of those features (i.e., the respective raster bandnames) are derived from the `config.ini` file and set at export time.

In this section, we read in the names from `config.ini` and compare that list with that from the feature raster assets. If the two do not match, it might be best to go back to the `featuresForModeling` module and fix it before proceeding here.

## Feature names from config file

Read-in, one by one, names of all features as stored in the config file.

**NOTE:** 
* This is done by manually scanning the config file. There is currently no way to do this programmatically. 
* `lon` and `lat` are not set in the config file, so are initialized here. Because they are not generated & saved as feature rasters but are simply invoked with `ee.Image.pixelLonLat()` at labeled points sampling time.

In [2]:
with open("config.ini", 'r') as f:
    fileContents = f.read()
config = cp.RawConfigParser(allow_no_value = True, interpolation = cp.ExtendedInterpolation())
config.read_file(io.StringIO(fileContents))

homeFolder = config.get('CORE', 'assetFolderWithFeatures')

configSeas = config['FEATURES-SEASONALITY']
phase      = configSeas.get('featureBandNamePhase')
ampl       = configSeas.get('featureBandNameAmplitude')
offset     = configSeas.get('featureBandNameOffset')
trend      = configSeas.get('featureBandNameTrend')
phase_seg  = configSeas.get('featureBandNamePhaseSegmented')
ampl_seg   = configSeas.get('featureBandNameAmplitudeSegmented')

configTCTd  = config['FEATURES-TASSELLEDCAP']
dBrt        = configTCTd.get('featureBandNameDrySeasonBrightness')
dGrn        = configTCTd.get('featureBandNameDrySeasonGreenness')
dWet        = configTCTd.get('featureBandNameDrySeasonWetness')
wBrt        = configTCTd.get('featureBandNameWetSeasonBrightness')
wGrn        = configTCTd.get('featureBandNameWetSeasonGreenness')
wWet        = configTCTd.get('featureBandNameWetSeasonWetness')
tctdiff     = configTCTd.get('featureBandNameTCTDifference')
dBrt_seg    = dBrt + '_seg'
dGrn_seg    = dGrn + '_seg'
dWet_seg    = dWet + '_seg'
wBrt_seg    = wBrt + '_seg'
wGrn_seg    = wGrn + '_seg'
wWet_seg    = wWet + '_seg'
tctdiff_seg = configTCTd.get('featureBandNameTCTDifferenceSegmented')

configGrBr   = config['FEATURES-GREENINGTEMPORALSTATS']
ndviMin      = configGrBr.get('featureBandNameGreening5thPerc')
ndviMed      = configGrBr.get('featureBandNameGreening50thPerc')
ndviMax      = configGrBr.get('featureBandNameGreening95thPerc')
grnExt       = configGrBr.get('featureBandNameGreeningExtent')
brnExt       = configGrBr.get('featureBandNameBrowningExtent')
grnbrnNd     = configGrBr.get('featureBandNameGrningBrningNd')
ndviMed_seg  = configGrBr.get('featureBandNameGreening50thPercSegmented')
grnExt_seg   = configGrBr.get('featureBandNameGreeningExtentSegmented')
brnExt_seg   = configGrBr.get('featureBandNameBrowningExtentSegmented')
grnbrnNd_seg = configGrBr.get('featureBandNameGrningBrningNdSegmented')

configPalsar = config['FEATURES-PALSAR']
palsar       = configPalsar.get('featureBandNamePalsarMedian')
palsar_seg   = configPalsar.get('featureBandNamePalsarMedianSegmented')

configPalsarScansar      = config['FEATURES-PALSAR-SCANSAR']
palsarScansar            = configPalsarScansar.get('featureBandNamePalsarScansarMedian')
palsarScansar_seg        = configPalsarScansar.get('featureBandNamePalsarScansarMedianSegmented')
palsarScansarRange       = configPalsarScansar.get('featureBandNamePalsarScansarRange')
palsarScansarRange_seg   = configPalsarScansar.get('featureBandNamePalsarScansarRangeSegmented')

configEt   = config['FEATURES-EVAPOTRANSPIRATION']
etSoil     = configEt.get('featureBandNameEtSoil')
etVeg      = configEt.get('featureBandNameEtVeg')
etSoil_seg = configEt.get('featureBandNameEtSoilSegmented')
etVeg_seg  = configEt.get('featureBandNameEtVegSegmented')

configGeomorph = config['FEATURES-TOPOGRAPHY-GEOMORPHOLOGY-RUGGEDNESS']
geomorphRugged = configGeomorph.get('featureBandName')
geomorphRugged_seg = configGeomorph.get('featureBandNameGeomorphRuggedSegmented')

configCanopyHeight   = config['FEATURES-CANOPY-HEIGHT']
cnpyheight           = configCanopyHeight.get('featureBandName')
cnpyheightStdDev     = configCanopyHeight.get('featureStdDevBandName')
cnpyheight_seg       = configCanopyHeight.get('featureBandNameCanopyHeightSegmented')
cnpyheightStdDev_seg = configCanopyHeight.get('featureBandNameCanopyHeightStdDevSegmented')
    
zoneNumSuff = 'Num'
zoneOheSuff = 'Ohe_'

configZnStates   = config['AOI-CLASSIFICATION-ZONES-STATES']
zoneStatePref    = configZnStates.get("featureBandNamePrefix")
zoneStateNum     = zoneStatePref + zoneNumSuff
zoneStateLabels  = list(ast.literal_eval(configZnStates.get("groupsOfStatesLabels")))
zoneStateOhe     = [zoneStatePref + zoneOheSuff + label for label in zoneStateLabels]

configZnBiomes  = config['AOI-CLASSIFICATION-ZONES-BIOMES']
zoneBiomePref   = configZnBiomes.get("featureBandNamePrefix")
zoneBiomeNum    = zoneBiomePref + zoneNumSuff
zoneBiomeLabels = list(ast.literal_eval(configZnBiomes.get("biomeLabels")))
zoneBiomeOhe    = [zoneBiomePref + zoneOheSuff + label for label in zoneBiomeLabels]

configZnGeolAge = config['AOI-CLASSIFICATION-ZONES-GEOLOGICAL-AGE']
zoneGeolAgePref = configZnGeolAge.get("featureBandNamePrefix")
zoneGeolAgeNum  = zoneGeolAgePref + zoneNumSuff
zoneGeolAgeLabels = list(ast.literal_eval(configZnGeolAge.get("geologicalAgeNames")))
zoneGeolAgeOhe    = [zoneGeolAgePref + zoneOheSuff + label for label in zoneGeolAgeLabels]

elev_seg = config.get('FEATURES-ELEVATION', 'featureBandNameElevSegmented')
ppt_seg = config.get('FEATURES-PRECIPITATION', 'featureBandNameAnnRainSegmented')
topoMtpi_seg = config.get('FEATURES-TOPOGRAPHY-MTPI', 'featureBandNameTopoMtpiSegmented')
topoHand_seg = config.get('FEATURES-TOPOGRAPHY-HAND', 'featureBandNameTopoHandSegmented')
aoi = config.get('AOI', 'bandNameAOI')

lon = "longitude"
lat = "latitude"

Gather them all into a list of lists, where sublists are conceptually grouped set of features.

**NOTE:** 
* The nesting should be one-deep, no more, no less
* Classification zones feature/s appear differently in the list, depending on whether they have numerical or one-hot encoding
  * Numeric: `[zoneNum]`, because it is a single band raster
  * One-hot: `zoneOhe`, because it is a multiband raster, and `zoneOhe` is already a list with all the bands

In [3]:
def flattenList(nestedList):
    fl = []
    for sublist in nestedList:
        fl = fl + sublist
    return fl

gatheredFromConfig = [ \
    [phase, ampl, offset, trend, phase_seg, ampl_seg], \
    [dBrt, dGrn, dWet, wBrt, wGrn, wWet, tctdiff, dBrt_seg, dGrn_seg, dWet_seg, wBrt_seg, wGrn_seg, wWet_seg, tctdiff_seg], \
    [ndviMin, ndviMed, ndviMax, grnExt, brnExt, grnbrnNd, ndviMed_seg, grnExt_seg, brnExt_seg, grnbrnNd_seg], \
    [palsar, palsar_seg], \
    [palsarScansar, palsarScansar_seg, palsarScansarRange, palsarScansarRange_seg], \
    [etSoil, etVeg, etSoil_seg, etVeg_seg], \
    [elev_seg], \
    [ppt_seg], \
    [topoMtpi_seg, topoHand_seg], \
    [geomorphRugged, geomorphRugged_seg], \
    [cnpyheight, cnpyheightStdDev, cnpyheight_seg, cnpyheightStdDev_seg], \
    [zoneStateNum], \
    zoneStateOhe, \
    [zoneBiomeNum], \
    zoneBiomeOhe, \
    [zoneGeolAgeNum], \
    zoneGeolAgeOhe, \
    [lon, lat], \
    [aoi]]

fromConfigFlattened = flattenList(gatheredFromConfig)

## Feature names from saved feature rasters

The function `assembleAllExistingFeatureRasters()` in the `generateFeatures` module gathers all relevant rasters from the config-designated folder of EE assets.

It "manually" reads-in all feature rasters using their generation functions, but with `returnExisting` flag set to `True`.


In [4]:
fromFeatureFolder = generateFeatures.assembleAllExistingFeatureRasters().bandNames()

## Compare & check for full match

In [5]:
print("Features from config file:", fromConfigFlattened)
print("Features from asset rasters:", fromFeatureFolder.getInfo())
print("Asset folder for rasters:", config.get('CORE', 'assetFolderWithFeatures'))

print("Full match between the two (True/False):", ee.List(fromConfigFlattened).containsAll(fromFeatureFolder).getInfo())

Features from config file: ['phase', 'amplitude', 'offset', 'trend', 'phase_seg', 'amplitude_seg', 'd_brt', 'd_grn', 'd_wet', 'w_brt', 'w_grn', 'w_wet', 'tct_diff', 'd_brt_seg', 'd_grn_seg', 'd_wet_seg', 'w_brt_seg', 'w_grn_seg', 'w_wet_seg', 'tct_diff_seg', 'mn', 'md', 'mx', 'grng_ext', 'brng_ext', 'nd_grbr', 'md_seg', 'grng_ext_seg', 'brng_ext_seg', 'nd_grbr_seg', 'palsar', 'palsar_seg', 'palsar_scansar_median', 'palsar_scansar_median_seg', 'palsar_scansar_range', 'palsar_scansar_range_seg', 'et_soil', 'et_veg', 'et_soil_seg', 'et_veg_seg', 'elev_seg', 'annrf_seg', 'topo_seg', 'topo_hand_seg', 'ruggedness', 'ruggedness_seg', 'canopyHeight', 'canopyHeightStdDev', 'canopyHeight_seg', 'canopyHeightStdDev_seg', 'zoneStateNum', 'zoneStateOhe_pbbh', 'zoneStateOhe_rj', 'zoneStateOhe_gj', 'zoneStateOhe_mh', 'zoneStateOhe_mpod', 'zoneStateOhe_aptg', 'zoneStateOhe_tn', 'zoneStateOhe_ka', 'zoneBiomeNum', 'zoneBiomeOhe_na', 'zoneBiomeOhe_fldgrssvn', 'zoneBiomeOhe_mntgrsshrb', 'zoneBiomeOhe_trsub

## In case check fails...

... it means that updates to either the config file or raster set handled by `assembleAllExistingFeatureRasters()` has not been made to reflect in the other. 

Check for minor mistakes in either one or both of them.

OR

Check whether a new feature was recently added to the module and if it was done thoroughly:
* A separate section for it added in the `config.ini`, where its feature names are set
  * And it added to the [appropriate section here at the top here](#Feature-names-from-config-file)
* A separate function for it added in the `generateFeatures` module, that follows the standard structure as others with `returnExisting` and `startFreshExport` handling
  * Followed by its at least one successful execution with `startFreshExport = True`
* It getting added to the `assembleAllExistingFeatureRasters()`, with `returnExisting = True`

## [ **CLASSIFY CASE** ] GBT classifier, "global" model, containing all the good features we have

### Biomes zones OHE

#### Non-hierarchical

Set up the classifier and the predictor variables to use. Specify and create a folder for results, if it doesn't already exist. 

In [6]:
cOpts_gbt200 = dict(classifier = "GradientBoostedTrees", numTrees = 200, trainFraction = 0.7)
globalModelGbtBiomesZonesOhe = [ \
                 [phase_seg, ampl_seg], \
                 [palsarScansar_seg, palsarScansarRange_seg], \
                 [ndviMed_seg, grnExt_seg, brnExt_seg, grnbrnNd_seg], \
                 [topoHand_seg], \
                 [topoMtpi_seg], \
                 [dBrt_seg, dGrn_seg, dWet_seg, wBrt_seg, wGrn_seg, wWet_seg, tctdiff_seg], \
                 [ppt_seg], \
                 [elev_seg], \
                 [geomorphRugged_seg], \
                 [cnpyheight_seg, cnpyheightStdDev_seg], \
                 zoneBiomeOhe]
globalModelGbtBiomesZonesOheFlattened = flattenList(globalModelGbtBiomesZonesOhe)
fOpts_globalModelGbt_biomeZonesOhe = dict(names = globalModelGbtBiomesZonesOheFlattened, zonationBasis = "biomes", zoneEncodingMode = "oneHot")

# kaBoundBox = ee.Feature(ee.Geometry.Polygon(
#         [[[72.74853170247881, 18.65873128670056],
#           [72.74853170247881, 11.342174273000648],
#           [78.90087545247881, 11.342174273000648],
#           [78.90087545247881, 18.65873128670056]]]), \
#         {"system:index": "0"});

resFolderGlobalModelGbtBiomeZonesOhe = "globalModelGbt_BiomeZonesOhe_hier"
createResFolderCmd = f"earthengine create folder {homeFolder}{resFolderGlobalModelGbtBiomeZonesOhe}"
process = subprocess.Popen(createResFolderCmd.split(), stdout=subprocess.PIPE)
folderCreated, error = process.communicate()
print("Result folder creation error (if any):", folderCreated, error)

Result folder creation error (if any): b'Asset projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier already exists.\n' None


Run the non-hierarchical classification. Here, all 12 labels of interest are taken together and classification is done in a "flat" (non-hierarchical) manner.

Labels at a coarser-level can be taken to be implicitly evident from this 12-class map, and can be computed by aggregating subsets of the 12 classes appropriately according to a rule of hierarchy. We call this way of arriving at a hierarchical map as ***Implicit Hierarchy***.

In [8]:
globalModelGbtBiomesZonesOheRes = classifyAndAssess.trainAndPredict(fOpts_globalModelGbt_biomeZonesOhe, cOpts_gbt200, resultNewFolderName = resFolderGlobalModelGbtBiomeZonesOhe, startFreshExport = True)
# globalModelGbtBiomesZonesOheRes = classifyAndAssess.trainAndPredict(fOpts_globalModelGbt_biomeZonesOhe, cOpts_gbt200, roiForClassification = kaBoundBox, resultNewFolderName = resFolderGlobalModelGbtBiomeZonesOhe, startFreshExport = True)
# print(globalModelGbtBiomesZonesOheRes.getInfo())

#### Hierarchical

We experimented with multiple ways of creating hierarchical maps:
* ***Implicit Hierarchy***: First create a 12-class level-2 flat (i.e., non-hierarchical) map and then compute its corresponding level-1 labels by aggregating appropriate subsets according to the rule of hierarchy. Classification is performed non-hierarchically at level-2, and level-1 labels are *implicitly* derived from them.
* ***Dependent Hierarchy***: First create a 2-class level-1 label probabilities map. Then, append those probabilities to the features composite/vectors and run a classifier to create a 12-class map at level-2. Classification at level-2 is *dependent* on the classification outcome at level-1.
* ***Explicit Hierarchy***: This approach is based on work described in [*Gavish et al. (2018)*](https://doi.org/10.1016/j.isprsjprs.2017.12.002) Represent the label hierarchy as a tree. Run a classifier at each node of the tree using only its child labels (along with a correction to account for rest of the labels). The probabilistic outputs of all these classifiers are combined in the following two ways to arrive at labels and probabilities for level-2. The corresponding level-1 labels and probabilities are then computed by aggregating appropriate labels according to the label hierarchy.
  * Multiplicative
  * Stepwise

The code below produces dependent and explicit hierarchical maps (implicit hierarchy is the same as in the previous cell). After careful evaluation of these, we chose Explicit-Multiplicative, as our final map result. 

##### Explicit hierarchy

This is adapted from the methods described in [*Gavish et al.*](https://doi.org/10.1016/j.isprsjprs.2017.12.002). It runs a classifier and computes outputs at each of the label hierarchy tree nodes.

In [7]:
hierarchOpts_exp = dict(hierarchyMode = "explicit", coarserLevelLabelColumn = "labelL1", finerLevelLabelColumn = "label_2024") #  label_palsarHarmonised
globalModelGbtBiomesZonesOheRes_heir_master_explicit = classifyAndAssess.trainAndPredictHierarchical_master( \
    hierarchOpts_exp, \
    fOpts_globalModelGbt_biomeZonesOhe, \
    cOpts_gbt200, \
    resultNewFolderName = resFolderGlobalModelGbtBiomeZonesOhe, \
    startFreshExport = True)


['phase_seg', 'amplitude_seg', 'palsar_scansar_median_seg', 'palsar_scansar_range_seg', 'md_seg', 'grng_ext_seg', 'brng_ext_seg', 'nd_grbr_seg', 'topo_hand_seg', 'topo_seg', 'd_brt_seg', 'd_grn_seg', 'd_wet_seg', 'w_brt_seg', 'w_grn_seg', 'w_wet_seg', 'tct_diff_seg', 'annrf_seg', 'elev_seg', 'ruggedness_seg', 'canopyHeight_seg', 'canopyHeightStdDev_seg', 'zoneBiomeOhe_na', 'zoneBiomeOhe_fldgrssvn', 'zoneBiomeOhe_mntgrsshrb', 'zoneBiomeOhe_trsubtrmoistblfor', 'zoneBiomeOhe_tmpconiffor', 'zoneBiomeOhe_tempblmixdfor', 'zoneBiomeOhe_trsubtrconiffor', 'zoneBiomeOhe_mngr', 'zoneBiomeOhe_desxershrb', 'zoneBiomeOhe_trsubtrdryblfor', 'zoneBiomeOhe_trsubtrgrssvnshrb']
l0 names & codes: {'nonone': 100, 'one': 200}
l0 hist: {'100': 236950, '200': 146900}
l1 names & codes: {'agri_hiBiomass': 1, 'agri_loBiomass': 2, 'built': 4, 'forest': 6, 'other': 9999, 'water_wetland': 12}
l1 hist: {'1': 15212, '12': 17276, '2': 156658, '4': 15140, '6': 32664, '9999': 146900}
l1 names & codes: {'bare': 3, 'dune':

##### Dependent hierarchy

The 2-class classification output of level-1 labels get appended to features composite, and level-2 classes are classified.

In [7]:
l1ProbsImage = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_l0")
hierarchOpts_dep = dict(hierarchyMode = "dependent", l1Probabilities = l1ProbsImage, coarserLevelLabelColumn = "labelL1", finerLevelLabelColumn = "label_2024")

globalModelGbtBiomesZonesOheRes_heir_master_dependent = classifyAndAssess.trainAndPredictHierarchical_master( \
    hierarchOpts_dep, \
    fOpts_globalModelGbt_biomeZonesOhe, \
    cOpts_gbt200, \
    resultNewFolderName = resFolderGlobalModelGbtBiomeZonesOhe, \
    startFreshExport = True)


['phase_seg', 'amplitude_seg', 'palsar_scansar_median_seg', 'palsar_scansar_range_seg', 'md_seg', 'grng_ext_seg', 'brng_ext_seg', 'nd_grbr_seg', 'topo_hand_seg', 'topo_seg', 'd_brt_seg', 'd_grn_seg', 'd_wet_seg', 'w_brt_seg', 'w_grn_seg', 'w_wet_seg', 'tct_diff_seg', 'annrf_seg', 'elev_seg', 'ruggedness_seg', 'canopyHeight_seg', 'canopyHeightStdDev_seg', 'zoneBiomeOhe_na', 'zoneBiomeOhe_fldgrssvn', 'zoneBiomeOhe_mntgrsshrb', 'zoneBiomeOhe_trsubtrmoistblfor', 'zoneBiomeOhe_tmpconiffor', 'zoneBiomeOhe_tempblmixdfor', 'zoneBiomeOhe_trsubtrconiffor', 'zoneBiomeOhe_mngr', 'zoneBiomeOhe_desxershrb', 'zoneBiomeOhe_trsubtrdryblfor', 'zoneBiomeOhe_trsubtrgrssvnshrb']
['prob_nonone', 'prob_one']
hierarchy mode dependent


##### Map outputs and accuracies

For all the approaches, this takes level-2 results and compute their corresponding level-1 results. It also computes accuracy metrics for each of these cases, to enable comparison among them.

In [7]:
explicitHierL1Probs = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_l0")
explicitHierL2NononeProbs = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_nonone")
explicitHierL2OneProbs = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_one")
dependentHierL2Result = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_l2Dep")
implicitHierL2Result = ee.Image("projects/ee-open-natural-ecosystems/assets/homeStretch2023/globalModelGbt_BiomeZonesOhe_hier/prediction_l2Flat")

combinedOutputs = classifyAndAssess.calcCoarserLevelLabelsAndAllAccuracies(explicitHierL1Probs, explicitHierL2NononeProbs, explicitHierL2OneProbs, dependentHierL2Result, implicitHierL2Result, resultNewFolderName = resFolderGlobalModelGbtBiomeZonesOhe)
# print(combinedOutputs.first().getInfo())